## Preparation (mapping isoforms back to genes)

In [76]:
import os
import csv
import pandas as pd
import re
import subprocess
import numpy as np
import scipy.stats as stats

new_dir = "/home/jingqi/RNALocateV3.0"
os.chdir(new_dir)
os.getcwd()

'/home/jingqi/RNALocateV3.0'

### Mapping

In [21]:
input_file = "Data/Raw/FASTA_full.txt"
output_file = "Data/Raw/gene_transcript_matrix.csv"

print(f"Reading from: {input_file}")

gene_to_transcripts = {}

with open(input_file, 'r') as f:
    for line in f:
        if line.startswith(">"):

            clean_line = line.strip()[1:] 
            parts = clean_line.split('|')
            
            if len(parts) >= 2:
                transcript_id = parts[0]
                gene_symbol = parts[1]
                
                # Add to dictionary
                if gene_symbol not in gene_to_transcripts:
                    gene_to_transcripts[gene_symbol] = []
                gene_to_transcripts[gene_symbol].append(transcript_id)

max_transcripts = 0
rows = []

for gene, transcripts in gene_to_transcripts.items():
    if len(transcripts) > max_transcripts:
        max_transcripts = len(transcripts)
    
    row = [gene] + transcripts
    rows.append(row)


print(f"Found {len(rows)} unique genes.")
print(f"Max transcripts for one gene: {max_transcripts}")

with open(output_file, "w", newline="") as f:
    writer = csv.writer(f)
    
    header = ["Gene_Symbol"] + [f"Transcript_{i+1}" for i in range(max_transcripts)]
    writer.writerow(header)
    
    writer.writerows(rows)


Reading from: Data/Raw/FASTA_full.txt
Found 728 unique genes.
Max transcripts for one gene: 102
------------------------------
Success! Matrix saved to: Data/Raw/gene_transcript_matrix.csv


### little validation (can skip)

In [32]:
final_gene_list = []
with open("Data/Raw/final_gene_list.txt", "r") as f:
    for line in f:
        gene = line.strip()
        if gene:
            final_gene_list.append(gene)

original_set = set(final_gene_list)

matrix_df = pd.read_csv("Data/Raw/gene_transcript_matrix.csv")
found_set = set(matrix_df['Gene_Symbol'].unique())

missing_genes = original_set - found_set
print(missing_genes)

736
728
7
{'ENSMUSG00000125456', 'ENSMUSG00000121480', 'ENSMUSG00000143377', 'ENSMUSG00000132887', 'ENSMUSG00000135935', '2810004N23Rik', 'ENSMUSG00000124710'}


## Assign SCL on to isoforms

### Extract the motifs validated

In [33]:
df_longer = pd.read_csv("Data/TOMTOM/tomtom_longer_filtered.csv")
df_shorter = pd.read_csv("Data/TOMTOM/tomtom_shorter_filtered.csv")

# Get unique target motif IDs
targets_longer = set(df_longer['Target'].dropna().unique())
targets_shorter = set(df_shorter['Target'].dropna().unique())
all_targets = targets_longer | targets_shorter

print(f"Unique targets from longer: {len(targets_longer)}")
print(f"Unique targets from shorter: {len(targets_shorter)}")
print(f"Total unique targets: {len(all_targets)}")

Unique targets from longer: 57
Unique targets from shorter: 149
Total unique targets: 188

Targets: ['mmu-miR-7086-5p', 'mmu-miR-6966-5p', 'mmu-miR-6919-3p', 'mmu-miR-124-3p', 'mmu-miR-204-5p', 'mmu-miR-6940-5p', 'mmu-miR-326-5p', 'mmu-miR-6968-5p', 'mmu-miR-12196-3p', 'mmu-miR-3090-5p']...


### Get the seqeunces from database

In [42]:
def extract_motifs_from_meme_db(db_file, target_ids, output_file):
    """
    Extract specific motifs from a MEME database file.
    """
    with open(db_file, 'r') as f:
        content = f.read()
    
    # Split by MOTIF
    parts = re.split(r'(?=^MOTIF )', content, flags=re.MULTILINE)
    
    # Keep header (first part before any MOTIF)
    header = parts[0] if not parts[0].startswith('MOTIF') else ""
    
    # Find motifs matching the target IDs
    kept_motifs = []
    for part in parts:
        if part.startswith('MOTIF'):
            # Extract motif ID (first word after MOTIF)
            match = re.match(r'MOTIF\s+(\S+)', part)
            if match:
                motif_id = match.group(1)
                if motif_id in target_ids:
                    kept_motifs.append(part)
    
    # Write subset file
    with open(output_file, 'w') as f:
        f.write(header)
        for motif in kept_motifs:
            f.write(motif)
    
    print(f"\n Saved {len(kept_motifs)} motifs to {output_file}")
    return len(kept_motifs)

# Path to original database (same one used for TOMTOM)
databases = {
    "CISBP_RNA": "~/meme_db/motif_databases/CISBP-RNA/Mus_musculus.dna_encoded.meme",
    "RNA": "~/meme_db/motif_databases/RNA/Ray2013_rbp_Mus_musculus.dna_encoded.meme", 
    "miRBase": "~/meme_db/motif_databases/MIRBASE/22/Mus_musculus_mmu.dna_encoded.meme",
}

# Extract subset
os.makedirs("Data/FIMO", exist_ok=True)

for db_name, db_path in databases.items():
    full_path = os.path.expanduser(db_path)
    output_path = f"Data/FIMO/filtered_{db_name}.meme"
    extract_motifs_from_meme_db(full_path, all_targets, output_path)




 Saved 19 motifs to Data/FIMO/filtered_CISBP_RNA.meme

 Saved 4 motifs to Data/FIMO/filtered_RNA.meme

 Saved 165 motifs to Data/FIMO/filtered_miRBase.meme


### Run FIMO (all transcripts against identified motifs)

In [45]:
fimo_path = "/home/jingqi/miniconda3/envs/rnalocate_old/bin/fimo"
motif_file = "Data/FIMO/all_filtered_motifs.meme"
fasta_file = "Data/Raw/FASTA.txt"
output_dir = "Data/FIMO/result_all_transcripts"

fimo_cmd = [
    fimo_path,
    "--oc", output_dir,
    "--thresh", "0.0001",
    motif_file, 
    fasta_file 
]

subprocess.run(fimo_cmd, check=True)

Using motif +M004_0.6 of width 7.
Using motif -M004_0.6 of width 7.
Computing q-values.
Estimating pi_0 from a uniformly sampled set of 10000 p-values.
Estimating pi_0.
Estimated pi_0=0.991031
Using motif +M033_0.6 of width 8.
Using motif -M033_0.6 of width 8.
Computing q-values.
Estimating pi_0 from a uniformly sampled set of 10000 p-values.
Estimating pi_0.
Estimated pi_0=1
Using motif +M042_0.6 of width 7.
Using motif -M042_0.6 of width 7.
Computing q-values.
Using motif +M043_0.6 of width 7.
Using motif -M043_0.6 of width 7.
Computing q-values.
Estimating pi_0 from a uniformly sampled set of 10000 p-values.
Estimating pi_0.
Estimated pi_0=0.930196
Using motif +M044_0.6 of width 7.
Using motif -M044_0.6 of width 7.
Computing q-values.
Estimating pi_0 from a uniformly sampled set of 10000 p-values.
Estimating pi_0.
Estimated pi_0=0.775926
Using motif +M049_0.6 of width 7.
Using motif -M049_0.6 of width 7.
Computing q-values.
Estimating pi_0 from a uniformly sampled set of 10000 p-val

CompletedProcess(args=['/home/jingqi/miniconda3/envs/rnalocate_old/bin/fimo', '--oc', 'Data/FIMO/result_all_transcripts', '--thresh', '0.0001', 'Data/FIMO/all_filtered_motifs.meme', 'Data/Raw/FASTA.txt'], returncode=0)

### Blend in SCL

In [70]:
# CORRECT HEADER
fimo_cols = [
    "motif_id", "sequence_name", "start", "stop", "strand", 
    "score", "p-value", "q-value", "matched_sequence"
]

fimo_df = pd.read_csv(
    "Data/FIMO/result_all_transcripts/fimo.txt", 
    sep='\t', 
    comment='#', 
    header=None, 
    names=fimo_cols
)

filtered_df = fimo_df[fimo_df['q-value'] < 0.05]

print(f"n\ Hits with q-value < 0.05: {len(filtered_df)}")

gene_lookup = {}

df = pd.read_csv("Data/Raw/gene_transcript_matrix.csv")

# Iterate row by row to build the dictionary
for index, row in df.iterrows():
    gene_symbol = row['Gene_Symbol']
    transcripts = row.drop('Gene_Symbol').dropna()
    
    for transcript in transcripts:
        if isinstance(transcript, str):
            ids = transcript.strip()
            gene_lookup[ids] = gene_symbol

print(f"Loaded {len(gene_lookup)} gene-transcript mappings")
print(f"Unique Genes found: {df['Gene_Symbol'].nunique()}")

df_longer = pd.read_csv("Data/TOMTOM/tomtom_longer_filtered.csv")
df_shorter = pd.read_csv("Data/TOMTOM/tomtom_shorter_filtered.csv")

# Combine and create mapping class, bucket
motif_to_cluster = {}

for _, row in df_longer.iterrows():
    target = row['Target']
    cluster = f"{row['Class']}_{row['Bucket']}"
    if target not in motif_to_cluster:
        motif_to_cluster[target] = cluster

for _, row in df_shorter.iterrows():
    target = row['Target']
    cluster = f"{row['Class']}_{row['Bucket']}"
    if target not in motif_to_cluster:
        motif_to_cluster[target] = cluster

print(f"Mapped {len(motif_to_cluster)} motifs to clusters")


prob_df = pd.read_csv("Data/Main/Probabilities.csv")
prob_df = prob_df.set_index('sequence_id')

prob_cols = {
    'chromatin': 'chromatin_prob',
    'cytoplasm': 'cytoplasm_prob',
    'cytosol': 'cytosol_prob',
    'ER': 'endoplasmic reticulum_prob',
    'extracellular': 'extracellular region_prob',
    'membrane': 'membrane_prob',
    'mitochondrion': 'mitochondrion_prob',
    'nucleolus': 'nucleolus_prob',
    'nucleoplasm': 'nucleoplasm_prob',
    'nucleus': 'nucleus_prob',
    'ribosome': 'ribosome_prob'
}

results = []

for _, row in filtered_df.iterrows():
    motif_id = row['motif_id']
    transcript = row['sequence_name']
    p_value = row['p-value']
    q_value = row['q-value']
    
    # Get gene for this transcript
    gene = gene_lookup.get(transcript, 'Unknown')
    
    # Get cluster for this motif
    cluster = motif_to_cluster.get(motif_id, None)
    if not cluster:
        continue
    
    # Parse class and bucket
    parts = cluster.rsplit('_', 1)
    if len(parts) != 2:
        continue
    class_name, bucket = parts
    
    # Get transcript probability
    if transcript not in prob_df.index:
        continue
    
    # Find the probability column
    prob_col = prob_cols.get(class_name, None)
    if not prob_col:
        for col in prob_df.columns:
            if class_name.lower() in col.lower():
                prob_col = col
                break
    
    if not prob_col:
        continue
    
    prob_value = prob_df.loc[transcript, prob_col]
    
    # Validate based on bucket
    if bucket == 'positive':
        expected = "High"
        match = prob_value > 0.5
    else:
        expected = "Low"
        match = prob_value < 0.5
    
    results.append({
        'Gene': gene,
        'Transcript': transcript,
        'Motif': motif_id,
        'Cluster': cluster,
        'Probability': prob_value,
        'Expected': expected,
        'Match': '✓' if match else '✗',
        'q_value': q_value
    })

result_df = pd.DataFrame(results)

# Sort: genes alphabetically, Unknown at the end
result_df['_sort_key'] = result_df['Gene'].apply(lambda x: (1, '') if x == 'Unknown' else (0, x.lower()))
result_df = result_df.sort_values(by=['_sort_key', 'Gene', 'Transcript'])
result_df = result_df.drop(columns=['_sort_key'])

# Reset index
result_df = result_df.reset_index(drop=True)

# Save
result_df.to_csv("Data/FIMO/motifs_validated.csv", index=False)
print(f"\n Total validated hits: {len(result_df)}")

# Keep only the validated matches (✓)
clean_df = result_df[result_df['Match'] == '✓'].copy()
clean_df = clean_df[result_df['Gene'] != 'Unknown']

# Drop the validation columns 
cols_to_drop = ['Probability', 'Expected', 'Match']
clean_df = clean_df.drop(columns=cols_to_drop, errors='ignore')

output_file = "Data/FIMO/motifs_validated_filtered.csv"
clean_df.to_csv(output_file, index=False)

if not result_df.empty:
    matches = (result_df['Match'] == '✓').sum()
    total = len(result_df)
    print(f"Matching predictions: {matches}/{total} ({100*matches/total:.1f}%)")
    
    # Gene stats
    print(f"Unique genes: {result_df[result_df['Gene'] != 'Unknown']['Gene'].nunique()}")
    
    print(f"\nBy cluster:")
    for cluster in sorted(result_df['Cluster'].unique()):
        subset = result_df[result_df['Cluster'] == cluster]
        m = (subset['Match'] == '✓').sum()
        t = len(subset)
        print(f"  {cluster}: {m}/{t} ({100*m/t:.1f}%)")


Loaded 5788 gene-transcript mappings
Unique Genes found: 728
Mapped 188 motifs to clusters

 Total validated hits: 50432
Matching predictions: 29750/50432 (59.0%)
Unique genes: 709

By cluster:


/tmp/ipykernel_3124637/2164612541.py:130: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clean_df = clean_df[result_df['Gene'] != 'Unknown']


  ER_negative: 1025/1049 (97.7%)
  ER_positive: 6/200 (3.0%)
  chromatin_negative: 62/122 (50.8%)
  chromatin_positive: 14230/29490 (48.3%)
  cytoplasm_negative: 142/322 (44.1%)
  cytoplasm_positive: 86/138 (62.3%)
  extracellular_negative: 0/18 (0.0%)
  extracellular_positive: 267/271 (98.5%)
  membrane_negative: 40/71 (56.3%)
  mitochondrion_negative: 412/1402 (29.4%)
  mitochondrion_positive: 14/19 (73.7%)
  nucleolus_negative: 14/24 (58.3%)
  nucleoplasm_negative: 1133/2522 (44.9%)
  nucleoplasm_positive: 979/1759 (55.7%)
  nucleus_positive: 11144/12271 (90.8%)
  ribosome_negative: 196/754 (26.0%)


### Compress by Motifs

In [82]:
# compress all the same motifs, check if one motif hits two or more localization

input_file = "Data/FIMO/motifs_validated_filtered.csv"
df = pd.read_csv(input_file)
# This converts "1.2e-5" strings to numbers
df['q_value'] = pd.to_numeric(df['q_value'])

#  Define Fisher's Method Function
def fisher_score(group):
    q_values = group['q_value'].values
    # Calculate Chi-Squared Statistic: -2 * sum(ln(q))
    chi2_stat = -2 * np.sum(np.log(q_values))
    # Degrees of freedom = 2 * number of tests
    df_chi2 = 2 * len(q_values)
    # Calculate combined p-value (Upper tail of Chi2 distribution)
    p_val = stats.chi2.sf(chi2_stat, df_chi2)
    return pd.Series({'Hits': len(q_values), 'Fisher_score': chi2_stat, 'p_adjusted': p_val})

# Filter out motifs that appear in more than one cluster
cluster_counts = df.groupby('Motif')['Cluster'].nunique()
valid_motifs = cluster_counts[cluster_counts == 1].index

# Filter the main dataframe to keep only these valid motifs
df_valid = df[df['Motif'].isin(valid_motifs)].copy()

print(f"  - Original unique motifs: {df['Motif'].nunique()}")
print(f"  - Motifs with conflicting clusters (discarded): {df['Motif'].nunique() - len(valid_motifs)}")
print(f"  - Motifs kept: {len(valid_motifs)}")

# Group by motif and cluster 
agg_df = df_valid.groupby(['Motif', 'Cluster']).apply(fisher_score).reset_index()

# Sort
agg_df = agg_df.sort_values('Fisher_score', ascending = False)

# Format the Output
agg_df = agg_df[['Motif', 'Cluster', 'Hits', 'Fisher_score', 'p_adjusted']]

output_file = "Data/FIMO/a_aggregated.csv"
agg_df.to_csv(output_file, index=False)

print(f"\n Aggregated file saved to: {output_file}")

  - Original unique motifs: 89
  - Motifs with conflicting clusters (discarded): 0
  - Motifs kept: 89

Success! Aggregated file saved to: Data/FIMO/motifs_aggregated.csv


### Compress by transcript (then motif)

In [91]:
# compress all the transcripts with the same motif, at least one hit being true is enough

df = pd.read_csv("Data/FIMO/motifs_validated_filtered.csv")

# Ensure numeric types and clean data
df['q_value'] = pd.to_numeric(df['q_value'])
df = df.dropna(subset=['q_value'])

def combined_pval(group):
    q_values = group['q_value'].values
    log_qs = np.log(q_values) 
    prob_all_false = np.exp(np.sum(log_qs))
    # Phred Score -logP
    phred_score = -(np.sum(log_qs) / np.log(10))
    return  pd.Series({
        'Counts': len(q_values),
        'p_adjusted': prob_all_false,
        'Phred_score': phred_score
    })

# We keep Site_Count and Best_Site_Q as useful secondary metrics
compressed_df = df.groupby(['Gene', 'Transcript', 'Motif', 'Cluster']).apply(combined_pval).reset_index()

# sorting, plus cluter the rows with the same gene together
compressed_df = compressed_df.sort_values(by=['Gene', 'Phred_score'], ascending=[True, False])
# compressed_df = compressed_df.sort_values('Phred_score', ascending = False)

output_file = "Data/FIMO/a_compressed_in_transcripts.csv"
compressed_df.to_csv(output_file, index=False)

print(f"Compressed {len(df)} hits into {len(compressed_df)} unique transcript-motif pairs.")
print(f"Results saved to: {output_file}")

Compressed 29720 hits into 10450 unique transcript-motif pairs.
Results saved to: Data/FIMO/a_compressed_in_transcripts.csv


### Compress by motif on transcript-compressed data (can skip)

In [93]:
# mostly like the cell above above

input_file = "Data/FIMO/a_compressed_in_transcripts.csv"
df = pd.read_csv(input_file)
# This converts "1.2e-5" strings to numbers
df['q_value'] = pd.to_numeric(df['p_adjusted'])

#  Define Fisher's Method Function
def fisher_score(group):
    q_values = group['q_value'].values
    # Calculate Chi-Squared Statistic: -2 * sum(ln(q))
    chi2_stat = -2 * np.sum(np.log(q_values + 10e-100))
    # Degrees of freedom = 2 * number of tests
    df_chi2 = 2 * len(q_values)
    # Calculate combined p-value (Upper tail of Chi2 distribution)
    p_val = stats.chi2.sf(chi2_stat, df_chi2)
    return pd.Series({'Hits': len(q_values), 'Fisher_score': chi2_stat, 'p_adjusted': p_val})

# Group by motif and cluster 
agg_df = df.groupby(['Motif', 'Cluster']).apply(fisher_score).reset_index()

# Sort
agg_df = agg_df.sort_values('Fisher_score', ascending = False)

# Format the Output
agg_df = agg_df[['Motif', 'Cluster', 'Hits', 'Fisher_score', 'p_adjusted']]

output_file = "Data/FIMO/a_aggregated_delicate.csv"
agg_df.to_csv(output_file, index=False)

print(f"\n Aggregated file saved to: {output_file}")

  - Original unique motifs: 89
  - Motifs with conflicting clusters (discarded): 0
  - Motifs kept: 89

Success! Aggregated file saved to: Data/FIMO/a_aggregated_delicate.csv


### Compress by cluster

In [99]:
df = pd.read_csv("Data/FIMO/motifs_validated_filtered.csv")
df_selected = df[['Gene', 'Transcript', 'Cluster']].drop_duplicates().copy()

# rule 1: Merge same transcript with same cluster
# Record the "Cluster Distribution" for each Transcript
tx_clusters = df_selected.groupby(['Gene', 'Transcript'])['Cluster'].apply(lambda x: tuple(sorted(x))).reset_index(name='Subcellular_Distribution')

# rule 2: Discard uniform genes
# Count the distribution sets for each gene
gene_dist_counts = tx_clusters.groupby('Gene')['Subcellular_Distribution'].nunique()

# Only keep genes with differential cluster distributions
valid_genes = gene_dist_counts[gene_dist_counts > 1].index
# Filter the dataframe
final_df = df_selected[df_selected['Gene'].isin(valid_genes)].copy()

# Sort for readability
final_df = final_df.sort_values(by=['Gene', 'Transcript', 'Cluster'])

output_file = "Data/FIMO/a_differential_isoforms.csv"
final_df.to_csv(output_file, index=False)

print(f"Saved to: {output_file}")
print(f"Genes kept: {len(valid_genes)}")
print(f"Final rows: {len(final_df)}")

Saved to: Data/FIMO/a_differential_isoforms.csv
Genes kept: 584
Final rows: 5953


### Comress by cluster (only keep positive distribution)

In [106]:
df = pd.read_csv("Data/FIMO/motifs_validated_filtered.csv")

# Before everything, filter for positive
df_positive = df[df['Cluster'].str.endswith('_positive')].copy()
# Remove the '_positive' suffix from the names
df_positive['Localization'] = df_positive['Cluster'].str.replace('_positive', '')

df_selected = df_positive[['Gene', 'Transcript', 'Localization']].drop_duplicates().copy()

# rule 1: Merge same transcript with same cluster
# Record the "Cluster Distribution" for each Transcript
tx_clusters = df_selected.groupby(['Gene', 'Transcript'])['Localization'].apply(lambda x: tuple(sorted(x))).reset_index(name='Subcellular_Distribution')

# rule 2: Discard uniform genes
# Count the distribution sets for each gene
gene_dist_counts = tx_clusters.groupby('Gene')['Subcellular_Distribution'].nunique()

# Only keep genes with differential cluster distributions
valid_genes = gene_dist_counts[gene_dist_counts > 1].index
# Filter the dataframe
final_df = df_selected[df_selected['Gene'].isin(valid_genes)].copy()

# Sort for readability
final_df = final_df.sort_values(by=['Gene', 'Transcript', 'Localization'])

output_file = "Data/FIMO/a_positive_differential_isoforms.csv"
final_df.to_csv(output_file, index=False)

print(f"Saved to: {output_file}")
print(f"Genes kept: {len(valid_genes)}")
print(f"Final rows: {len(final_df)}")

Saved to: Data/FIMO/a_positive_differential_isoforms.csv
Genes kept: 433
Final rows: 3208


In [107]:

df = pd.read_csv("Data/FIMO/a_positive_differential_isoforms.csv")

# 3. Create a sequence number for each localization on a transcript
df['Loc_Num'] = df.groupby(['Gene', 'Transcript']).cumcount() + 1
df['Loc_Num'] = 'Localization_' + df['Loc_Num'].astype(str)

# 4. Pivot the table. This takes the 'Loc_Num' column and turns its values into brand new column headers
df_wide = df.pivot(
    index=['Gene', 'Transcript'], 
    columns='Loc_Num', 
    values='Localization'
).reset_index()

# Clean up the internal column names to make it look nice
df_wide.columns.name = None

# Sort the dataframe for easy reading
df_wide = df_wide.sort_values(by=['Gene', 'Transcript'])

output_file = "Data/FIMO/a_genewise.csv"
df_wide.to_csv(output_file, index=False)

print(f"Success! Compressed down to {len(df_wide)} unique transcripts.")
print(f"Saved to: {output_file}")

Success! Compressed down to 1961 unique transcripts.
Saved to: Data/FIMO/a_genewise.csv
